In [10]:
import os
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score, balanced_accuracy_score

tmp_dataset = pd.read_csv('dataset/gender_info.csv')

training_folder = 'dataset/Testing'
testing_folder = 'dataset/Training'

print(tmp_dataset)
# add file paths to the audio files

training_files = os.listdir(training_folder)
testing_files = os.listdir(testing_folder)

train_dataset = pd.DataFrame()

for filename in training_files:
    name, extension = os.path.splitext(filename)
    df_name = tmp_dataset.columns['File Names'][name]
    item = tmp_dataset[df_name == name]
    train_dataset['path'] = training_folder + '/' + filename
    train_dataset['gender'] = item['Gender']
    train_dataset['filename'] = filename

test_dataset = pd.DataFrame()
for filename in testing_files:
    name, extension = os.path.splitext(filename)
    item = tmp_dataset[tmp_dataset['File Names'] == name]
    test_dataset['path'] = training_folder + '/' + filename
    test_dataset['gender'] = item['Gender']
    test_dataset['filename'] = filename

print(train_dataset)
print(test_dataset)

assert len(train_dataset) == len(training_files)
assert len(test_dataset) == len(testing_files)


            File Names  Gender
0       84-121123-0028  Female
1       84-121123-0027  Female
2       84-121123-0026  Female
3       84-121123-0025  Female
4       84-121123-0024  Female
...                ...     ...
1123  1272-128104-0004    Male
1124  1272-128104-0003    Male
1125  1272-128104-0002    Male
1126  1272-128104-0001    Male
1127  1272-128104-0000    Male

[1128 rows x 2 columns]
                                     path gender             filename
1127  dataset/Testing/84-121123-0028.flac    NaN  84-121123-0028.flac


KeyboardInterrupt: 

In [ ]:

# Function to extract features from an audio file
def extract_features(file_path):
    audio, sr = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)

    return np.hstack((
        np.mean(mfccs, axis=1),
        np.mean(spectral_centroids),
        np.mean(chroma, axis=1),
        np.mean(zero_crossing_rate)
    ))


# Load CSV file
df = pd.read_csv('dataset/gender_info.csv')

# Extract features
features = []
labels = []

for index, row in df.iterrows():
    file_path = os.path.join('dataset/Training', row['file_name'])
    features.append(extract_features(file_path))
    labels.append(row['gender'])

# Convert to numpy arrays
X = np.array(features)
y = np.array(labels)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a simple SVM model
model = SVC(kernel='rbf', random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Calculate metrics
f_score = f1_score(y_test, y_pred, average='weighted')
balanced_acc = balanced_accuracy_score(y_test, y_pred)

print(f"F-score: {f_score}")
print(f"Balanced Accuracy: {balanced_acc}")